## The System:

![grafik.png][def]


[def]: .\PTG_System.png

## Preperation
#### Welche Packages brauchen wir NICHT?

In [6]:
!pip install gurobipy==10.0.0
!pip install pyomo==6.6.2
!pip install numpy==1.24.3
!pip install pandas==1.3.5
!pip install tsam==2.3.1
!pip install matplotlib
!pip install july

In [16]:
from gurobipy import *
from pyomo.environ import *
#import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tsam.timeseriesaggregation as tsam
#import july
#from july.utils import date_range


## Time series
#### Daten laden

In [21]:
data_sheet = pd.read_excel(".\Data_Sheet_neu.xlsx")
#print(data_sheet)

time_slices = np.array(time_series_data.index)
#print(time_slices)

FileNotFoundError: [Errno 2] No such file or directory: '.\\Sourcetree_Daten\\Data_Sheet_neu.xlsx'

## Setting up the model:

In [22]:
model = ConcreteModel() #aus Pyomo

# declare sets
model.time_slices = set(time_slices) #was passiert hier genau? iwas mit Pyomo

model.typical_days_row = Set(typical_days_row) #das wäre eine Idee 
#typical_days_row=[1,2,3] # hier aus Time agregation die hier einfügen:
#   # Assign labels to each original period
#   labels = aggregation.clusterOrder
#   print("Cluster Labels:")
#   print(labels)

NameError: name 'time_slices' is not defined

In [ ]:
# declare decision variables
model.OPEX = Var(domain=NonNegativeReals) # [€]

# declare time dependent decision variables
model.P_PV = Var(model.time_slices, domain=NonNegativeReals) # Photovoltaic [kW], nötig?
model.P_PEM = Var(model.time_slices, domain=NonNegativeReals) # Electrolysis [kW]
model.CO2_DAC = Var(model.time_slices, domain=NonNegativeReals) # DirectAirCapture [kg_CO2/hr]
model.P_MR = Var(model.time_slices, domain=NonNegativeReals) # MethaneReformation [kW]
model.Qdot_CHP = Var(model.time_slices, domain=NonNegativeReals) # CombinedHeatPower [kW]
model.P_CHP = Var(model.time_slices, domain=NonNegativeReals) # CombinedHeatPower [kW]
model.Qdot_BO = Var(model.time_slices, domain=NonNegativeReals) # Boiler [MW]

model.P_battery = Var(model.time_slices, model.typical_days_row, domain=Reals) # [MW] [negative: storage charge, positive: storage discharge]
model.storage_level_battery = Var(model.time_slices, model.typical_days_row, domain=NonNegativeReals) # [MWh]
model.Qdot_thermalStorage = Var(model.time_slices, model.typical_days_row, domain=Reals) # [MW] [negative: storage charge, positive: storage discharge]
model.storage_level_thermalStorage = Var(model.time_slices, model.typical_days_row, domain=NonNegativeReals) # [MWh]
model.P_H2Storage = Var(model.time_slices, domain=Reals) # [MW] [negative: storage charge, positive: storage discharge]
model.storage_level_H2Storage = Var(model.time_slices, domain=NonNegativeReals) # [MWh]
model.P_CO2Storage = Var(model.time_slices, domain=Reals) # [MW] [negative: storage charge, positive: storage discharge]
model.storage_level_CO2Storage = Var(model.time_slices, domain=NonNegativeReals) # [MWh]
model.P_CH4Storage = Var(model.time_slices, domain=Reals) # [MW] [negative: storage charge, positive: storage discharge]
model.storage_level_CH4Storage = Var(model.time_slices, domain=NonNegativeReals) # [MWh]

In [ ]:
# declare parameters

#WAS IST DAS?
#i = 0.05
#lifetime = 30
#model.RBF = ((1 + i) ** lifetime * i) / ((1 + i) ** lifetime - 1)

model.P_N_PV = 900 #[kw_e]

model.P_N_PEM = 900 #[kW_e]
model.eta_N_PEM = 0.63 #eta muss noch linerarisiert werden!!

model.CO2_N_DAC = 250 #[kg_CO2/hr]
model.eta_DAC = 0.5 #[kg_CO2/kWh_e]
#ps fehlen noch

model.P_N_MR = 1500 #[kW_CH4]
model.eta_MR = 0.78

model.Q_N_CHP = 800 #[kW_Q]
model.eta__N_CHP_Q = 0.47 #eta muss noch linerarisiert werden!!
model.eta_N_CHP_P = 0.4 #eta muss noch linerarisiert werden!!
#ps fehlen noch

model.Q_N_BO = 500 #[kW]
model.eta_N_BO = 0.9 #eta muss noch linerarisiert werden!!
#ps fehlen noch

model.E_N_battery = 5000 #[kWh_e]
model.eta_battery = 0.92 
model.loss_battery = 0.000042 #[1/hr]
model.c_battery = 0.36 #[1/hr]

model.Q_N_thermalStorage = 5000 #[kWh_Q]
model.eta_thermalStorage = 0.95 
model.loss_thermalStorage = 0.005 #[1/hr]
model.c_thermalStorage = 1 #[1/hr]

model.E_N_H2Storage = 5000 #[kWh_H2]
model.m_N_CO2Storage = 5000 #[kg_CO2]
model.E_N_CH4Storage = 5000 #[kWh_CH4]
model.loss_gasTanks = 0 #[1/hr]
model.eta_gasTanks = 1 
model.c_gasTanks = 0.25 #[1/hr]


In [ ]:
# declare time dependent parameters
#model.time_slice_weight = Param(model.time_slices, initialize = time_slice_weight)
#model.gas_price = Param(model.time_slices, initialize = gas_price_time_series)
#model.electricity_price = Param(model.time_slices, initialize = electricity_price_time_series)
#model.Q_demand = Param(model.time_slices, initialize = heat_demand_time_series)

In [ ]:
# declare unindexed constraints



# declare rules to setup indexed constraints



# declare indexed constraints

## Solving the problem

In [ ]:
#SolverFactory('gurobi').solve(model).write()